In [1]:
import psfws
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import galsim
plt.rcParams['font.size']=14

## Simplest usage example

In [2]:
# set a seed for rng, if desired
seed = 12435

# instantiate a weather station, this is where one can specify data, correlations, etc. Here defaults are used.
ws = psfws.ParameterGenerator(seed=seed)

# if user wants to draw parameters from a random datapoint, use:
params = ws.draw_parameters()

# what's in here?
params.keys()

dict_keys(['u', 'v', 't', 'speed', 'phi', 'h', 'j', 'edges'])

These keys correspond to the following parameters:
- `h`: altitudes, in km above sea level, of the phase screens to be simulated. The rest of the parameters below are sampled at these altitudes.
- `speed`, `phi`: wind speed (m/s) and direction. `phi` is defined as the meteorological wind direction, i.e. the direction *from* which the blows, in degrees east of north.
- `j`: turbulence integrals J for each phase screen. These correspond to the `r0_weights` input for `galsim.Atmosphere()`.
- `u`, `v`: the components of wind speed in the E, N directions. Equivalent information to the speed and direction above. 
- `t`: temperature profile. Not required for galsim simulations.
- `edges`: the boundaries between each of the layers. Not required for simulations, but convenient for plotting.

To see these parameters plotted and to explore either the default or your own imported data, see the [exploring environmental parameters](./exploring-environmental-parameters.ipynb) notebook.

### Specifying datapoint for which to get parameters:

If user wants to specify a particular datapoint to draw from, use `ws.get_parameters()` instead, which takes an additional argument of a pandas TimeStamp. This value must be in the index of the data; i.e. must be in `ws.data_fa.index`

In [3]:
pt = ws.data_fa.index[34]
params = ws.get_parameters(pt)

## Running a simple galsim.Atmosphere() simulation with these parameters

Run a simulation at zenith, we will introduce coordinate complications next.

Parameters below set up a short exposure, few layer, and relatively small FoV simulation for quick run-time.

#### Setting up simulation parameters -- modify these as you like

In [8]:
# simulation parameters
FOV = 1
EXPTIME = 15.

NSCREEN = 5
NPHOT = int(1e5)
NPOOL = 4
NPSF = 1000

SCREENSIZE = 300
SCREENSCALE = 0.1

ATMSEED = 1
PSFSEED = 2

# atmosphere parameters
L0 = 25 # outer scale
lam = 754.06 # collection wavelength
r0_500 = .16 # Fried parameter as measured at lambda=500nm

#### Set up the simulation 

In [9]:
# as above, draw parameters from psf-weather-station given a seed and a number of screens.
ws = psfws.ParameterGenerator(seed=ATMSEED)
params = ws.draw_parameters(nl=NSCREEN)

# note that observatory altitude should be subtracted, since galsim takes 0 to be ground.
params['h'] = [p - ws.h0 for p in params['h']]

# define set of atmospheric phase screens
atm = galsim.Atmosphere(r0_500=r0_500,
                        altitude=list(params['h']),
                        L0=[L0]*NSCREEN,
                        r0_weights=list(params['j']),
                        speed=list(params['speed']),
                        direction=[d*galsim.degrees for d in params['phi']],
                        screen_size=SCREENSIZE,
                        screen_scale=SCREENSCALE,
                        rng=galsim.BaseDeviate(ATMSEED))

# telescope aperture: Rubin-like diameter/obscuration
aper = galsim.Aperture(diam=8.36, obscuration=0.61, lam=lam, screen_list=atm)

# r0 at simulation wavelength sets k-modes to include
r0 = r0_500 * (lam/500)**1.2
kcrit = 0.2
kmax = kcrit / r0

# instantiate phase screen list, for photon shooting simulation
atm.instantiate(kmax=kmax, check='phot')

print("done instantiating!")

done instantiating!


#### Run the simulation

In [10]:
# define empty arrays
thxs = np.empty(NPSF, dtype=float)
thys = np.empty(NPSF, dtype=float)
psfPhotSeeds = np.empty(NPSF, dtype=float)

# generate random uniform numbers into these arrays
psfRng = galsim.BaseDeviate(PSFSEED)
ud = galsim.UniformDeviate(psfRng)
ud.generate(thxs)
ud.generate(thys)
ud.generate(psfPhotSeeds)

# center the random values, and scale by the field of view -- 
# these will now be the theta x/y values for the positions of stars on the FoV
thxs -= 0.5
thys -= 0.5
thxs *= FOV
thys *= FOV

psfPhotSeeds *= 2**20
psfPhotSeeds = psfPhotSeeds.astype(np.int64)

### Additional realism: coordinates

_coming soon_...